<a href="https://colab.research.google.com/github/teatime77/xbrl-reader/blob/master/notebook/sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Matplotlibで日本語が表示できるようにします。

#### IPAフォントをインストールします。

In [0]:
!apt-get -y install fonts-ipafont-gothic

#### Matplotlibのフォントのキャッシュを再構築します。

In [0]:
import matplotlib
matplotlib.font_manager._rebuild()

#### <font color="red">キャッシュの再構築を有効にするために、ここでランタイムを再起動してください。</font>

### <font color="red">以下の中から予測したい項目のコメントをはずしてください。</font>

In [0]:
target = '売上高'
# target = '営業利益'
# target = '経常利益'
# target = '税引前純利益'

### <font color="red">グリッドサーチをする場合は、以下の変数の値をTrueにしてください。</font>

In [0]:
use_grid_search = False

### 選択した項目に対応するファイルをダウンロードします。

In [0]:
if   target == '売上高':
    ! wget http://lkzf.info/xbrl/data/2020-04-08/preprocess-uriage.pickle

elif target == '営業利益':
    ! wget http://lkzf.info/xbrl/data/2020-04-08/preprocess-eigyo.pickle

elif target == '経常利益':
    ! wget http://lkzf.info/xbrl/data/2020-04-08/preprocess-keijo.pickle

elif target == '税引前純利益':
    ! wget http://lkzf.info/xbrl/data/2020-04-08/preprocess-jun.pickle

### CatBoostをインストールします。

In [0]:
! pip install catboost

### 必要なライブラリをインポートします。

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

sns.set(font='IPAGothic')

### データを読み込みます。

In [0]:
import pickle

if   target == '売上高':
    file_name = 'preprocess-uriage.pickle'
elif target == '営業利益':
    file_name = 'preprocess-eigyo.pickle'
elif target == '経常利益':
    file_name = 'preprocess-keijo.pickle'
elif target == '税引前純利益':
    file_name = 'preprocess-jun.pickle'
else:
    assert False

with open(file_name, 'rb') as f:
    data = pickle.load(f)

df = data['data_frame']
y_column = data['y_column']

### トレーニングデータとテストデータに分けます。

In [0]:
X_columns = [ x for x in df.columns if x != y_column ]

# トレーニングデータとテストデータに分けます。
X_train, X_test, y_train, y_test = train_test_split(df[X_columns], df[y_column], test_size=0.2, random_state=0)

### モデルを実行します。

In [0]:
from sklearn.model_selection import GridSearchCV

def run_model(model):
    global y_test, y_pred

    if use_grid_search and type(model) in [ RandomForestRegressor, XGBRegressor, LGBMRegressor, CatBoostRegressor ]:
        # グリッドサーチをする場合

        model = GridSearchCV(model, {'max_depth': [2,4,6], 'n_estimators': [50,100,200]}, verbose=2, n_jobs=-1)

    # トレーニングデータで学習します。
    result = model.fit(X_train, y_train)

    if hasattr(result, 'best_params_'):
        # 最適なパラメータがある場合
        
        print ('best params =', result.best_params_)

    # テストデータで予測します。
    y_pred = model.predict(X_test)

    # 平均二乗誤差を計算します。
    accu1 = mean_squared_error(y_test, y_pred)
    accu2 = mean_squared_error(y_test, [y_test.mean()] * len(y_test) )

    # 平均絶対誤差を計算します。
    accu3 = mean_absolute_error(y_test, y_pred)
    accu4 = mean_absolute_error(y_test, [y_test.mean()] * len(y_test) )

    print('\n平均二乗誤差 : %.4f ( %.4f )  平均絶対誤差 : %.4f ( %.4f )    ※ カッコ内は全予測値を平均値で置き換えた場合\n' % (accu1, accu2, accu3, accu4))    

    if hasattr(model, 'feature_importances_'):
        # 特徴量の重要度がある場合
        
        # 重要度の順にソートします。
        sorted_idx_names = sorted(enumerate(model.feature_importances_), key=lambda x: x[1], reverse=True)

        print('特徴量の重要度')
        for i, (idx, x) in enumerate(sorted_idx_names[:20]):
            print('    %2d %.05f %s' % (i, 100 * x, X_train.columns[idx]))


    # 正解と予測を散布図に表示します。
    sns.jointplot(y_test, y_pred, kind="reg").set_axis_labels('正解', '予測')

### リッジ回帰

In [0]:
model = Ridge(alpha=.5)
run_model(model)

# 売上高の予測の場合、これくらいになるはず
# 平均二乗誤差 : 0.0645 ( 0.0667 )  平均絶対誤差 : 0.1784 ( 0.1968 )

### サポートベクターマシン

In [0]:
model = SVR(kernel='rbf')
run_model(model)

# 売上高の予測の場合、これくらいになるはず
# 平均二乗誤差 : 0.0517 ( 0.0667 )  平均絶対誤差 : 0.1670 ( 0.1968 )

### ランダムフォレスト

In [0]:
model = RandomForestRegressor(max_depth=6, n_estimators=200)
run_model(model)

# 売上高の予測の場合、これくらいになるはず
# 平均二乗誤差 : 0.0510 ( 0.0667 )  平均絶対誤差 : 0.1680 ( 0.1968 )

### XGBoost

In [0]:
model = XGBRegressor(max_depth=2, n_estimators=200)
run_model(model)

# 売上高の予測の場合、これくらいになるはず
# 平均二乗誤差 : 0.0496 ( 0.0667 )  平均絶対誤差 : 0.1642 ( 0.1968 )

### LightGBM

In [0]:
model = LGBMRegressor(objective='regression', num_leaves = 31, max_depth=4, n_estimators=50)
run_model(model)

# 売上高の予測の場合、これくらいになるはず
# 平均二乗誤差 : 0.0495 ( 0.0667 )  平均絶対誤差 : 0.1645 ( 0.1968 )

### CatBoost

In [0]:
model = CatBoostRegressor(max_depth=2, n_estimators=200, verbose=0)
run_model(model)

# 売上高の予測の場合、これくらいになるはず
# 平均二乗誤差 : 0.0500 ( 0.0667 )  平均絶対誤差 : 0.1646 ( 0.1968 )